In [19]:
# Import libraries
import cv2
import numpy as np
from mtcnn import MTCNN
from deepface import DeepFace
import os

In [3]:
# 1. Image Dimension Validation
def validate_dimensions(image_path, required_width=413, required_height=531):
    image = cv2.imread(image_path)
    if image is None:
        return False, "Invalid image file"
    height, width, _ = image.shape
    return (
        width == required_width and height == required_height,
        f"Dimensions should be {required_width}x{required_height} pixels",
    )


In [ ]:
def validate_background(image_path, threshold=180):
    image = cv2.imread(image_path)
    if image is None:
        return False, "Invalid image file"

    # Converting to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Create a mask for the background (assuming the background is the largest connected component)
    _, binary = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    background_mask = np.zeros_like(gray)
    cv2.drawContours(background_mask, contours, -1, 255, thickness=cv2.FILLED)

    # Calculate the percentage of white pixels in the background
    white_pixels = np.sum((gray > threshold) & (background_mask == 255))
    total_pixels = np.sum(background_mask == 255)
    white_percentage = (white_pixels / total_pixels) * 100
    print(f"White Percentage: {white_percentage:.2f}%")

    # Check if the background is predominantly white
    return white_percentage > 80, "Background should be predominantly white"


In [5]:
# 3. Face Detection and Positioning
def validate_face_positioning(image_path):
    detector = MTCNN()
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    if image is None:
        return False, "Invalid image file"

    results = detector.detect_faces(image)
    if not results:
        return False, "No face detected"

    face = results[0]
    keypoints = face["keypoints"]
    height, width, _ = image.shape
    face_height = face["box"][3]

    # Check if shoulders are visible
    if face_height > 0.7 * height:
        return False, "Shoulders not visible"

    # Check if ears are unobstructed
    if not all(keypoints.values()):
        return False, "Ears obstructed"

    return True, "Face positioning valid"


In [17]:
import traceback
# Emotion validation
def validate_emotion(image_path):
    try:
        print(f"Starting emotion validation on: {image_path}")
        # Check if file exists
        if not os.path.exists(image_path):
            return False, "Image file not found"

        # Analyze emotion
        attributes = DeepFace.analyze(
            img_path=image_path, actions=["emotion"], enforce_detection=False
        )

        print(f"DeepFace analysis completed")

        # Handle multiple faces
        if isinstance(attributes, list):
            if not attributes:
                return False, "No face detected for emotion analysis"
            # Use the first face in the list
            attributes = attributes[0]

        # Debug the emotion scores
        emotion_scores = attributes["emotion"]
        print(f"Emotion scores: {emotion_scores}")

        # Check for neutral expression
        if emotion_scores["neutral"] < 0.5:
            dominant_emotion = max(emotion_scores, key=emotion_scores.get)
            return False, f"Expression not neutral (detected: {dominant_emotion})"

        return True, "Expression is neutral"
    except Exception as e:
        print(f"Exception in validate_emotion: {str(e)}")
        print(traceback.format_exc())
        return False, f"Emotion validation failed: {str(e)}"


In [20]:
result, message = validate_emotion(image_path)
print("Emotion Validation:", result, message)


Starting emotion validation on: C:\Users\anves\Downloads\test2.jpg
DeepFace analysis completed
Emotion scores: {'angry': np.float32(0.583719), 'disgust': np.float32(4.4413217e-05), 'fear': np.float32(1.5197457), 'happy': np.float32(8.929203), 'sad': np.float32(4.2673764), 'surprise': np.float32(0.0028152522), 'neutral': np.float32(84.697105)}
Emotion Validation: True Expression is neutral


In [8]:
import cv2


def validate_head_covering(image_path):
    try:
        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            return False, "Invalid image file"

        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Load Haar Cascade for face detection
        face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
        )

        # Detect faces
        faces = face_cascade.detectMultiScale(
            gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30)
        )

        if len(faces) == 0:
            return False, "No face detected"

        # Get the first detected face
        (x, y, w, h) = faces[0]

        # Check the aspect ratio of the face
        aspect_ratio = w / h
        if aspect_ratio < 0.7 or aspect_ratio > 1.3:  # Adjust thresholds as needed
            return False, "Head covering detected (abnormal aspect ratio)"

        # Analyze the upper region of the face for head coverings
        upper_face = gray[y : y + h // 4, x : x + w]  # Upper 25% of the face
        upper_face_mean = upper_face.mean()

        if upper_face_mean < 100:  # Adjust threshold as needed
            return False, "Head covering detected (dark upper region)"

        return True, "No head covering detected"
    except Exception as e:
        return False, f"Head covering validation failed: {str(e)}"


In [21]:
# Test the functions
image_path = image_path = "C:\\Users\\anves\\Downloads\\test2.jpg"
print("Dimensions:", validate_dimensions(image_path))
print("Background:", validate_background(image_path))
print("Face Positioning:", validate_face_positioning(image_path))
result, message = validate_emotion(image_path)
print("Emotion Validation:", result, message)
head_covering_result, head_covering_message = validate_head_covering(image_path)
print("Head Covering Validation:", head_covering_result, head_covering_message)


Dimensions: (False, 'Dimensions should be 413x531 pixels')
White Percentage: 99.84%
Background: (np.True_, 'Background should be predominantly white')
Face Positioning: (True, 'Face positioning valid')
Starting emotion validation on: C:\Users\anves\Downloads\test2.jpg
DeepFace analysis completed
Emotion scores: {'angry': np.float32(0.583719), 'disgust': np.float32(4.4413217e-05), 'fear': np.float32(1.5197457), 'happy': np.float32(8.929203), 'sad': np.float32(4.2673764), 'surprise': np.float32(0.0028152522), 'neutral': np.float32(84.697105)}
Emotion Validation: True Expression is neutral
Head Covering Validation: False Head covering detected (dark upper region)


In [ ]:
image_path = "C:\\Users\\anves\\Downloads\\picture.jpg"
print("Image Path:", image_path)


Image Path: C:\Users\anves\Downloads\picture.jpg


In [23]:
# Test the functions
image_path = image_path = "C:\\Users\\anves\\Downloads\\testimage1.jpg"
print("Dimensions:", validate_dimensions(image_path))
print("Background:", validate_background(image_path))
print("Face Positioning:", validate_face_positioning(image_path))
result, message = validate_emotion(image_path)
print("Emotion Validation:", result, message)
head_covering_result, head_covering_message = validate_head_covering(image_path)
print("Head Covering Validation:", head_covering_result, head_covering_message)


Dimensions: (False, 'Dimensions should be 413x531 pixels')
White Percentage: 99.80%
Background: (np.True_, 'Background should be predominantly white')
Face Positioning: (True, 'Face positioning valid')
Starting emotion validation on: C:\Users\anves\Downloads\testimage1.jpg
DeepFace analysis completed
Emotion scores: {'angry': np.float32(98.49446), 'disgust': np.float32(0.016209887), 'fear': np.float32(0.21796107), 'happy': np.float32(0.00048769385), 'sad': np.float32(0.27402943), 'surprise': np.float32(0.7810714), 'neutral': np.float32(0.2157687)}
Emotion Validation: False Expression not neutral (detected: angry)
Head Covering Validation: True No head covering detected


In [ ]:
# Test the functions
image_path = image_path = "C:\\Users\\anves\\Downloads\\closeup.jpg"
print("Dimensions:", validate_dimensions(image_path))
print("Background:", validate_background(image_path))
print("Face Positioning:", validate_face_positioning(image_path))
result, message = validate_emotion(image_path)
print("Emotion Validation:", result, message)
head_covering_result, head_covering_message = validate_head_covering(image_path)
print("Head Covering Validation:", head_covering_result, head_covering_message)


Dimensions: (False, 'Dimensions should be 413x531 pixels')
White Percentage: 84.31%
Background: (np.True_, 'Background should be predominantly white')
Face Positioning: (True, 'Face positioning valid')
Starting emotion validation on: C:\Users\anves\Downloads\smile1.jpg
DeepFace analysis completed
Emotion scores: {'angry': np.float32(3.9986108e-09), 'disgust': np.float32(2.787883e-13), 'fear': np.float32(2.1554867e-07), 'happy': np.float32(99.93808), 'sad': np.float32(7.606305e-08), 'surprise': np.float32(0.0054144203), 'neutral': np.float32(0.05651099)}
Emotion Validation: False Expression not neutral (detected: happy)
Head Covering Validation: True No head covering detected
